In [1]:
# Import C++ perceptron implementation
import neuralnet as nn
# Import other libraries
import matplotlib.pyplot as plt
import numpy as np
import pickle
import time
np.random.seed(123)

In [2]:
# Load training data
print("Loading pickled data")
TRAIN_IN, TRAIN_OUT = pickle.load(open('../data/mnist_preprocessed.pickle', 'rb'))
N = 2
a = 0.00001
AdamArgs = (0.9, 0.999)

In [3]:
model = nn.Sequence.new(
    nn.TanH.new(784, 500, *AdamArgs), 
    nn.ReLU.new(500, 300, *AdamArgs), 
    nn.ReLU.new(300, 100, *AdamArgs), 
    nn.ReLU.new(100, 100, *AdamArgs), 
    nn.ReLU.new(100, 50, *AdamArgs), 
    nn.SoftMax.new(50, 10, *AdamArgs)
)
trainer = nn.Trainer(model, TRAIN_IN, TRAIN_OUT)

In [ ]:
ts = time.time()
errors = trainer.train(N, a)
duration = time.time() - ts
print(f"Training of static model complete in {duration / N} seconds per epoch.")

plt.title("Training Error")
plt.plot(range(len(errors)), errors)
plt.show()

: 